In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as f
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import lightgbm as lgbm
from torch.utils.data import DataLoader
from sklearn import preprocessing
from tqdm import tqdm
from sklearn.preprocessing import FunctionTransformer
import seaborn as sns 
from scipy import stats
import statsmodels.api as sm

from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor, BaggingRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import ElasticNet, Lasso
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import SVR
from sklearn.model_selection import KFold, cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA
from sklearn.preprocessing import RobustScaler
import h2o
from h2o.automl import H2OAutoML
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.neural_network import MLPRegressor
from scipy.stats import norm
import copy
from sklearn.model_selection import KFold
from catboost import CatBoostRegressor, Pool, metrics, cv
import xgboost as xgb
from scipy.stats import gmean
from sklearn.impute import KNNImputer
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
import optuna
from xgboost import XGBRegressor
from optuna import Trial, visualization
from sklearn.ensemble import BaggingRegressor
def metric(y_true, y_pred):
    return np.mean(np.sum(np.abs(y_true - y_pred), axis=0)/np.sum(y_true, axis=0))


In [2]:
train_df =  pd.read_csv('sub_train.csv')
test_df  =  pd.read_csv('sub_test.csv')
for c in train_df.columns:
    if train_df[c].dtypes != 'float64':
        print(c)

startdate
climateregions__climateregion


In [3]:
exclude_cols = ['index', 'startdate']
target=["contest-tmp2m-14d__tmp2m"]
main_attrs = [c for c in train_df.columns if c not in exclude_cols and c not in target]
print("Main features:", len(main_attrs))

Main features: 59


In [4]:
X = train_df[main_attrs].values
y = train_df[target[0]].values
Xtest = test_df[main_attrs].values

X.shape, y.shape, Xtest.shape

((375734, 59), (375734,), (31354, 59))

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import StackingRegressor

xgb = XGBRegressor()
lgb = lgbm.LGBMRegressor()
cat = CatBoostRegressor(iterations=4900, verbose=0)

final_estimator = GradientBoostingRegressor(n_estimators=200, 
                                            random_state=42)
estimators = [('xgb', XGBRegressor(verbosity=2, n_estimators=300, eta = 0.04057043057515679, subsample=0.7, min_child_weight=6, colsample_bytree=0.7, max_depth=13)),
              ('lgb', lgbm.LGBMRegressor()),  
              ('cat', CatBoostRegressor(verbose=0, iterations=4900))] 
reg = StackingRegressor(
        estimators=estimators,
         final_estimator=final_estimator)
kf = KFold(n_splits=5, shuffle=True, random_state=0)
scores = np.abs(cross_val_score(reg, X, y, cv=kf, scoring='neg_root_mean_squared_error'))
scores = [np.sqrt(-c) for c in scores]

print(f"Scores - {scores}\nMean - {sum(scores)/len(scores)}")

[21:43:33] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 3764 extra nodes, 0 pruned nodes, max_depth=13
[21:43:36] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 3840 extra nodes, 0 pruned nodes, max_depth=13
[21:43:36] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 3886 extra nodes, 0 pruned nodes, max_depth=13
[21:43:37] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 4016 extra nodes, 0 pruned nodes, max_depth=13
[21:43:38] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 3980 extra nodes, 0 pruned nodes, max_depth=13
[21:43:39] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 3976 extra nodes, 0 pruned nodes, max_depth=13
[21:43:40] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 4098 extra nodes, 0 pruned nodes, max_depth=13
[21:43:42] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 4062 extra nodes, 0 pruned nodes, max_depth=13
[21:43:44] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 401

[21:44:44] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 7658 extra nodes, 0 pruned nodes, max_depth=13
[21:44:45] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 6654 extra nodes, 0 pruned nodes, max_depth=13
[21:44:46] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 7458 extra nodes, 0 pruned nodes, max_depth=13
[21:44:48] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 7238 extra nodes, 0 pruned nodes, max_depth=13
[21:44:49] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 6506 extra nodes, 0 pruned nodes, max_depth=13
[21:44:50] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 8052 extra nodes, 0 pruned nodes, max_depth=13
[21:44:51] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 7678 extra nodes, 0 pruned nodes, max_depth=13
[21:44:52] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 7286 extra nodes, 0 pruned nodes, max_depth=13
[21:44:53] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 766

[21:48:17] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 6238 extra nodes, 0 pruned nodes, max_depth=13
[21:48:21] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 5664 extra nodes, 0 pruned nodes, max_depth=13
[21:48:22] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 5100 extra nodes, 0 pruned nodes, max_depth=13
[21:48:28] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 6056 extra nodes, 0 pruned nodes, max_depth=13
[21:48:30] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 5998 extra nodes, 0 pruned nodes, max_depth=13
[21:48:32] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 6694 extra nodes, 0 pruned nodes, max_depth=13
[21:48:38] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 5700 extra nodes, 0 pruned nodes, max_depth=13
[21:48:42] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 5126 extra nodes, 0 pruned nodes, max_depth=13
[21:48:47] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 590

[21:54:00] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 5420 extra nodes, 0 pruned nodes, max_depth=13
[21:54:08] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 3020 extra nodes, 0 pruned nodes, max_depth=13
[21:54:15] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 2700 extra nodes, 0 pruned nodes, max_depth=13
[21:54:22] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 3252 extra nodes, 0 pruned nodes, max_depth=13
[21:54:27] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 2562 extra nodes, 0 pruned nodes, max_depth=13
[21:54:31] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 3934 extra nodes, 0 pruned nodes, max_depth=13
[21:54:36] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 3288 extra nodes, 0 pruned nodes, max_depth=13
[21:54:41] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 4728 extra nodes, 0 pruned nodes, max_depth=13
[21:54:47] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 273

[22:02:47] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 1986 extra nodes, 0 pruned nodes, max_depth=13
[22:02:57] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 3348 extra nodes, 0 pruned nodes, max_depth=13
[22:03:05] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 3864 extra nodes, 0 pruned nodes, max_depth=13
[22:03:13] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 3244 extra nodes, 0 pruned nodes, max_depth=13
[22:03:21] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 1244 extra nodes, 0 pruned nodes, max_depth=13
[22:03:28] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 2298 extra nodes, 0 pruned nodes, max_depth=13
[22:03:35] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 2344 extra nodes, 0 pruned nodes, max_depth=13
[22:03:44] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 2802 extra nodes, 0 pruned nodes, max_depth=13
[22:03:52] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 187

[22:23:48] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 6354 extra nodes, 0 pruned nodes, max_depth=13
[22:23:59] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 6440 extra nodes, 0 pruned nodes, max_depth=13
[22:24:11] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 6780 extra nodes, 0 pruned nodes, max_depth=13
[22:24:19] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 6274 extra nodes, 0 pruned nodes, max_depth=13
[22:24:26] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 6448 extra nodes, 0 pruned nodes, max_depth=13
[22:24:35] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 6528 extra nodes, 0 pruned nodes, max_depth=13
[22:24:42] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 7000 extra nodes, 0 pruned nodes, max_depth=13
[22:24:52] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 7010 extra nodes, 0 pruned nodes, max_depth=13
[22:25:00] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 647

[22:32:20] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 5664 extra nodes, 0 pruned nodes, max_depth=13
[22:32:20] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 6356 extra nodes, 0 pruned nodes, max_depth=13
[22:32:21] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 6214 extra nodes, 0 pruned nodes, max_depth=13
[22:32:21] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 6426 extra nodes, 0 pruned nodes, max_depth=13
[22:32:23] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 5874 extra nodes, 0 pruned nodes, max_depth=13
[22:32:24] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 5864 extra nodes, 0 pruned nodes, max_depth=13
[22:32:25] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 5472 extra nodes, 0 pruned nodes, max_depth=13
[22:32:27] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 6296 extra nodes, 0 pruned nodes, max_depth=13
[22:32:28] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 659

In [12]:
print(f"Scores - {scores}\nMean - {sum(scores)/len(scores)}")

Scores - [nan, nan, nan, nan, nan]
Mean - nan
